In [1]:
import sys
sys.path.append('/source/main')

In [2]:
import logging
logging.basicConfig(level=logging.INFO)
import time
from itertools import chain

import torch
from torch import nn
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)

from model_def.baseline import Baseline
from utils import pytorch_utils
from data_for_train import my_dataset
from train import trainer

In [3]:
my_dataset.bootstrap()

INFO:root:Src vocab contains 3519 tokens
INFO:root:Tgt vocab contains 8758 tokens


In [4]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
# del model
model = Baseline(src_word_vocab_size=len(my_dataset.voc_src.index2word),
                     tgt_word_vocab_size=len(my_dataset.voc_tgt.index2word))
model.eval()
logging.info('Model architecture: \n%s', model)
logging.info('Total trainable parameters: %s', pytorch_utils.count_parameters(model))

INFO:root:Model architecture: 
Baseline(
  (input_embedding): Embedding(3519, 512)
  (conv1): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3_bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.3)
  (conv4): Conv1d(1024, 512, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv4_bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv1d(512, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv5_bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv6_bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=256, out_features=8758, bias=True)
  (relu)

In [5]:
MAX_LEN = 100

In [6]:
def docs2input_tensors(preprocessed_docs):
    seq_len = [len(doc.split()) for doc in preprocessed_docs]
    word_input = [my_dataset.voc_src.docs2idx([doc], equal_length=MAX_LEN)[0] for doc in preprocessed_docs]
    
    inputs = (word_input, seq_len)
    inputs = [np.array(i) for i in inputs]
    input_tensors = [torch.from_numpy(i) for i in inputs]
    return input_tensors

def replace_unk_tok(pred, src):
    pred = [p if p!='¶' else s for p, s in zip(pred, src)]
    return ''.join(pred)

def predict_batch(docs):
    input_tensors = docs2input_tensors(docs)
    predict_tensor = model.cvt_output(model(*input_tensors))
    predict_numpy = predict_tensor.cpu().numpy()
    
    translated_docs = my_dataset.voc_tgt.idx2docs(predict_numpy)
#     translated_docs = [pred[:len(src)] for src, pred in zip(docs, translated_docs)]
    translated_docs = [replace_unk_tok(pred, src) for pred, src in zip(translated_docs, docs)]
    return translated_docs

def predict_docs(docs, batch_size):
    return list(chain(*[predict_batch(docs[i: i+batch_size]) for i in range(0, len(docs), batch_size)]))


In [7]:
predict_docs(['hom nay toi di hoc'], batch_size=10)

['ôm tiffany quèn qu']

In [8]:
def get_metrics(df):
    logging.info('Total sentences: %s', df.shape[0])
    sen_acc = (df['tgt'] == df['pred']).sum()/df.shape[0]
    
    df = df[df['tgt'].map(lambda x: len(x.split())) == df['pred'].map(lambda x: len(x.split()))]
    logging.info('Total predicted sequences without changing len: %s', df.shape[0])
    tok_tgt = [tok for doc in df['tgt'] for tok in doc.split()]
    tok_pred = [tok for doc in df['pred'] for tok in doc.split()]
    sen_tok = (np.array(tok_tgt) == np.array(tok_pred)).sum()/len(tok_tgt)
    
    return sen_acc, sen_tok

# Predict

In [17]:
df = pd.read_csv('/source/main/data_for_train/output/my_test.csv')
df = df.iloc[:5000, :]

In [18]:
start = time.time()
pred = predict_docs(list(df['src']), batch_size=128)
end = time.time()
df['pred'] = pred
logging.info('Duration: %.2f s' % (end-start))

INFO:root:Duration: 53.54 s


In [19]:
get_metrics(df)

INFO:root:Total sentences: 5000
INFO:root:Total predicted sequences without changing len: 5


(0.0, 0.0)

In [20]:
df.head()

,tgt,src,pred
0,"các học trò của đàm vĩnh hưng , dù có người gọi anh là sư phụ , đại ca hay papa thì đều được vị huấn luyện này chăm cho từng chút , từ việc hướng dẫn thí sinh cách hát và chọn bài , tư vấn cách ăn mặc , giao tiếp , trả lời báo giới , báo giá cát-sê cho đến việc tự tay làm đẹp cho thí sinh .","cac hoc tro cua dam vinh hung , du co nguoi goi anh la su phu , dai ca hay papa thi deu duoc vi huan luyen nay cham cho tung chut , tu viec huong dan thi sinh cach hat va chon bai , tu van cach an mac , giao tiep , tra loi bao gioi , bao gia cat-se cho den viec tu tay lam dep cho thi sinh .",tiffany tiffany quèn tiffany tiffany dnnvv ips tiffany quèn tiffany quèn tiffany ips tiffany dnnvv tiffany tiffany quèn ips ips quèn tiffany tiffany tiffany tiffany quèn quèn quèn dnnvv ips keo ips ips tiffany tiffany quèn tiffany tiffany tiffany tiffany quèn dnnvv ips tiffany ips tiffany t
1,"( tno ) lầu năm góc đang tăng cường hệ thống phòng thủ tên lửa sau những lời đe dọa tấn công hạt nhân từ chdcnd triều tiên và chuẩn bị triển khai numpatt tên lửa đánh chặn ở hai bang alaska và california , theo fox news vào hôm nay , numpatt","( tno ) lau nam goc dang tang cuong he thong phong thu ten lua sau nhung loi de doa tan cong hat nhan tu chdcnd trieu tien va chuan bi trien khai numpatt ten lua danh chan o hai bang alaska va california , theo fox news vao hom nay , numpatt",tiffany quèn tiffany tiffany quèn dnnvv tiffany tiffany quèn tiffany quèn tiffany dnnvv tiffany quèn tiffany quèn quèn quèn quèn tiffany tiffany dnnvv quèn ấp dnnvv tiffany quèn quèn ôm dnnvv tiffany dnnvv tiffany quèn quèn tiffany tiffany q
2,"qua kết quả chụp x-quang , các bác sĩ đã chẩn đoán bệnh nhân bị hội chứng suy hô hấp cấp tính tiến triển , nghi ngờ viêm phổi do cúm a . ngay sau đó , bệnh nhân đã được lấy bệnh phẩm xét nghiệm cúm a/h1n1 , h5n1 và được đưa vào phòng cách ly và tiến hành các biện pháp cấp cứu .","qua ket qua chup x-quang , cac bac si da chan doan benh nhan bi hoi chung suy ho hap cap tinh tien trien , nghi ngo viem phoi do cum a . ngay sau do , benh nhan da duoc lay benh pham xet nghiem cum a/h1n1 , h5n1 va duoc dua vao phong cach ly va tien hanh cac bien phap cap cuu .",tiffany ips quèn tiffany ips quèn tiffany quèn quèn dnnvv tiffany quèn ips quèn soan quèn tiffany tiffany tiffany tiffany quèn quèn tiffany dnnvv tiffany tiffany tiffany tiffany quèn quèn tiffany ips quèn ips ips ips tiffany quèn ôm nhốt keo tiffany tiffany tiffany quèn ips tif
3,"đối với các nhà trường , ban giám hiệu nhất là hiệu trưởng phải thực sự thấy việc tổ chức các buổi sinh hoạt chuyên môn theo nghiên cứu bài học là cần thiết để nâng cao chất lượng giáo dục của nhà trường .","doi voi cac nha truong , ban giam hieu nhat la hieu truong phai thuc su thay viec to chuc cac buoi sinh hoat chuyen mon theo nghien cuu bai hoc la can thiet de nang cao chat luong giao duc cua nha truong .",tiffany tiffany ips tiffany ips khieu dnnvv ips quèn ips tiffany quèn tiffany cpu quèn quèn quèn ips tiffany tiffany tiffany tiffany tiffany quèn cảnh ips dnnvv ips ips tiffany ips tiffany ips quèn mg ips
4,"internet và di động đã thay đổi hoạt động mại dâm tại những nước như ấn độ , khiến cho ngành nghề này ngày càng giống như một `` dịch vụ mà khái niệm `` lao động tình dục '' ( sex worker ) được nhiều nước thừa nhận .","internet va di dong da thay doi hoat dong mai dam tai nhung nuoc nhu an do , khien cho nganh nghe nay ngay cang giong nhu mot `` dich vu ma khai niem `` lao dong tinh duc '' ( sex worker ) duoc nhieu nuoc thua nhan .",quèn tiffany quèn quèn tiffany ips dnnvv múa quèn tiffany ips mg dnnvv keo lẻ quèn ips quèn tiffany ips hũ quèn tiffany quèn ips ips dnnvv ips tiffany ips dnnvv ips tiffany quèn quèn keo quèn tiffany ips tiffany tiff
